In [ ]:
import pandas as pd
import numpy as np
import copy, time
import math
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score, KFold ,train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

df_train = pd.read_csv('./train_data.csv')
df_test = pd.read_csv('./test_features.csv')
print(df_train.shape)
columns=list(df_train.columns)
print(columns)
df_train.head()

In [ ]:
train_Y=df_train['poi'] #目標欄位
name = df_test['name'] 
df_train = df_train.drop(['name','poi'] , axis=1) 
columns.remove('name')
columns.remove('poi')
df_test = df_test.drop(['name'] , axis=1)
df = pd.concat([df_train, df_test],sort=False) #資料一併做前處理
df.head()

 'director_fees', 'exercised_stock_options', 'expenses', 'long_term_incentive','restricted_stock', 'restricted_stock_deferred',  'total_payments', 'total_stock_value' 

In [ ]:
#df['BAXTER JOHN C'].loc[] = 300000
#df['LAY KENNETH L'].loc = 

#### 缺失值處理

In [ ]:
def na_num(df_data):
    num = (df_data.isnull().sum()/len(df)).sort_values(ascending=False)
    return num

print(na_num(df)) #欄位缺失率

#缺失值比率過高，先捨去欄位
df = df.drop(['loan_advances'],axis=1)

#類別缺失值處理，改以是否有信箱
none_cols = ['email_address']
for col in none_cols:
    df[col]=df[col].isnull() 
    #df[col] = df[col].fillna("None")

#部分欄位缺失值填補0
zero_cols = ['long_term_incentive', 'deferred_income','deferral_payments','director_fees',\
            'from_messages','to_messages','from_poi_to_this_person', 'from_this_person_to_poi',\
            'shared_receipt_with_poi',\
             'director_fees', 'exercised_stock_options', 'expenses', 'long_term_incentive','restricted_stock',\
             'restricted_stock_deferred', 'total_payments', 'total_stock_value' ]
for col in zero_cols:
    df[col] = df[col].fillna(0)

#部分欄位缺失值填補眾數
mod_cols = []
for col in mod_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

#部分欄位缺失值填補平均數
mean_cols = ['other','exercised_stock_options']
for col in mean_cols:
    df[col] = df[col].fillna(df[col].mean())

#部分欄位缺失值填補中位數
med_cols = ['salary','bonus']
for col in med_cols:
    df[col] = df[col].fillna(df[col].median())

#確認缺失值填補是否完整
print(na_num(df)) 

#### 離群值處理

In [ ]:
#離群值處理
def remove_outlier(df):
    for k in df.columns[1:]:
        df_temp = df[k].copy()
        df_temp_iqr = iqr(df_temp)
        df_temp_std = np.std(df_temp)
        df_temp_med= np.median(df_temp)
        df_temp_q1=np.percentile(df_temp , 25)
        df_temp_q3=np.percentile(df_temp , 75)
        df_temp_lower = df_temp_q1-1.5*df_temp_iqr
        df_temp_upper = df_temp_q3+2*df_temp_iqr
        #df[k].fillna(df[k].median(),inplace=True)    
        df[k].loc[df_temp > df_temp_upper]=np.percentile(df_temp, q = 99.5)

#### Scale

In [ ]:
# 將資料最大最小化
#df = MinMaxScaler().fit_transform(df)
#將資料標準化
df = StandardScaler().fit_transform(df)

# 將前述轉換完畢資料 df , 重新切成 train_X, test_X
train_num = train_Y.shape[0]
train_X = pd.DataFrame(df[:train_num])
test_X = pd.DataFrame(df[train_num:])

train_Y = pd.DataFrame(train_Y.map(lambda x :int(x)))

#### 切分數據集

In [ ]:
kf=KFold(n_splits = 5, shuffle = True, random_state = 42)
result=next(kf.split(train_X), None)
train=train_X.iloc[result[0]]
valid=train_X.iloc[result[1]]
df_train=train
df_train_Y=train_Y.iloc[result[0]]
df_valid=valid
df_valid_Y=train_Y.iloc[result[1]]

df_train, df_valid, df_train_Y,df_valid_Y = train_test_split(train_X, train_Y,\
test_size=0.2, random_state=42)

In [ ]:
# 使用模型 : 邏輯斯迴歸 / Lasso / Ridge / 梯度提升 / 隨機森林
from sklearn.linear_model import LogisticRegression,Ridge,Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
#from sklearn.model_selection import GridSearchCV
t = lambda x :1 if x >1 else x
def tra(y):
    for i in range(len(y)):
        y[i]=t(y[i])
    return y

In [ ]:
import xgboost as xgb
param = {'max_depth':5, 'eta':1, 'objective':'binary:logistic', 'alpha': 0.0001, 'lambda': 1}
num_round = 15
dtrain = xgb.DMatrix(df_train,label=df_train_Y)
xgbst = xgb.train(param, dtrain, num_round)

dval= xgb.DMatrix(df_valid,label=df_valid_Y)

xgbst_predval = xgbst.predict(dval)
validlabel = dval.get_label()
AUC = metrics.roc_auc_score(validlabel,xgbst_predval)
print(AUC)
# make prediction
dtrain = xgb.DMatrix(train_X,label=train_Y)
xgbst = xgb.train(param, dtrain, num_round)
dtest = xgb.DMatrix(test_X)
xgbst_preds = xgbst.predict(dtest)
print(xgbst_preds)
sub = pd.DataFrame({'name': name, 'poi': xgbst_preds})
print(sub)
#sub.to_csv('Enron_xgboost.csv', index=False) 

In [ ]:
ridge = Ridge(alpha=.3)

###########train###########
ridge.fit(df_train,df_train_Y)
###########valid###########
ridge_predval =ridge.predict(df_valid).reshape(23)
print(ridge_predval)
AUC = metrics.roc_auc_score(df_valid_Y, ridge_predval)
print(AUC)

###########tset###########
#ridge.fit(train_X, train_Y)
ridge_pred = abs(ridge.predict(test_X)).reshape(33)

sub = pd.DataFrame({'name': name, 'poi': ridge_pred})
print(sub)
#sub.to_csv('Enron_ridge.csv', index=False) 

In [ ]:
lasso = Lasso(alpha=0.01)
###########train###########
lasso.fit(df_train,df_train_Y)
###########valid###########
lasso_predval = tra(abs(lasso.predict(df_valid)))
AUC=metrics.roc_auc_score(df_valid_Y, lasso_predval)
print(AUC)
###########tset###########
#lasso = Lasso(alpha=0.001)
#lasso.fit(train_X, train_Y)
lasso_pred = abs(lasso.predict(test_X))
sub = pd.DataFrame({'name': name, 'poi': tra(lasso_pred)})
print(sub)
#sub.to_csv('Enron_lasso.csv', index=False) 

In [ ]:
# 梯度提升機預測檔 
gdbt = GradientBoostingRegressor(tol=0.1, subsample=0.37, n_estimators=500, #max_features=20, 
                                 max_depth=8, learning_rate=0.01)
###########train###########
gdbt.fit(df_train,df_train_Y)
###########valid###########
gdbt_predval = tra(abs(gdbt.predict(df_valid)))
AUC=metrics.roc_auc_score(df_valid_Y, gdbt_predval)
print(AUC)

###########tset###########
gdbt.fit(train_X, train_Y)
gdbt_pred = abs(gdbt.predict(test_X))

sub = pd.DataFrame({'name': name, 'poi': tra(gdbt_pred)})
#print(sub)
#sub.to_csv('Enron_gdbt.csv', index=False)

In [ ]:
# 隨機森林預測檔 
rf = RandomForestRegressor(n_estimators=200, min_samples_split=8, min_samples_leaf=10,#max_features='', 
                           max_depth=6, bootstrap=False)
###########train###########
rf.fit(df_train,df_train_Y)
###########validation###########
rf_predval = abs(rf.predict(df_valid))
AUC=metrics.roc_auc_score(df_valid_Y, rf_predval)
print(AUC)
###########test###########
rf.fit(train_X, train_Y)
rf_pred = abs(rf.predict(test_X))
sub = pd.DataFrame({'name': name, 'poi': tra((rf_pred))})
#print(sub)
#sub.to_csv('Enron_rf.csv', index=False)

In [ ]:
log_r=LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0)
log_r.fit(df_train,df_train_Y)
log_r_predval = abs(log_r.predict_proba(df_valid))
log_r_pred=[]
for i,p in enumerate(log_r_predval):
    log_r_pred.append(log_r_predval[i][1])
log_r_pred=np.array(log_r_pred)
print(log_r_pred)
AUC=metrics.roc_auc_score(df_valid_Y, log_r_pred)
print(AUC)
###########test###########
log_r=LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0)
log_r.fit(train_X, train_Y)
log_r_pred=log_r.predict_proba(test_X)
print(log_r_pred)
pred=[]
for i,p in enumerate(log_r_pred):
    pred.append(log_r_pred[i][1])
pred=np.array(pred)
sub = pd.DataFrame({'name': name, 'poi':pred})
print(sub)
#sub.to_csv('Enron_logistic.csv', index=False)

In [ ]:
# 混合泛化預測
blending_pred = ridge_pred*0.05+ rf_pred*0.75+xgbst_preds*0.2
sub = pd.DataFrame({'name': name, 'poi':tra(blending_pred)})
#sub['poi'] = sub['poi'].map(lambda x:True if x>0.5 else False) 
print(sub)
#sub.to_csv('rf-xgbst-ridge.csv', index=False)